In [ ]:
%load_ext autoreload

%autoreload 2

import albumentations as A


from dataset import *

import numpy as np

import torch
import torchvision.models as models
from torchvision.models import EfficientNet_B3_Weights

import torch.nn as nn

from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import LabelEncoder

from utils import *
from code_eval import code

import scipy.io as sio

size_img = 300


data_transforms_albumentation = {
    'test': A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(size_img, size_img),
        ToTensorV2(),
    ])
}


model_file = "model_final.pth"

In [ ]:
torch.cuda.set_device(2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

print("Loading datasets")

train = sio.loadmat("train_data.mat")


test = sio.loadmat("test_data.mat")


train_images = train["images"]
test_images = test["images"]



labels_train = train["irma_code"]
labels_test = test["irma_code"]


irma_train = np.vstack(train["irma_code"][0])[:,0]
irma_test = np.vstack(test["irma_code"][0])[:,0]

test_images = np.array(test_images)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(irma_train)
test_labels = label_encoder.transform(irma_test)


evaluator = code("codes.txt")

model = models.efficientnet_b3(weights = EfficientNet_B3_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = True
model.classifier = nn.Sequential(nn.Dropout(
    0.2), nn.Linear(in_features=1536, out_features=193))
model.to(device)

test_dataset = ImagesDataset(
    test_images, test_labels, irma_test, transform=data_transforms_albumentation["test"], albumentation=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False, num_workers=0)

criterion = nn.CrossEntropyLoss()

state_dict = torch.load(model_file)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

model.eval()
print(validate_irma(model, test_loader, evaluator, label_encoder, device))



In [ ]:
from utils import *

probs = extract_softmax(model, test_loader)


#k and the prob threshold 0.7 were selected based on grid search described in the paper.
top_k = torch.tensor(np.vstack(probs)).topk(axis = 1, k = 2)[1]
top_k_probs = torch.tensor(np.vstack(probs)).topk(axis = 1, k = 2)[0]

irma_error_merging = 0
for i in range(0,len(probs)):
    top_k_labels = np.matrix(list(map( lambda n: list(label_encoder.inverse_transform([n])[0]),list(top_k[i]))))

    if(top_k_probs[i][0] < 0.7):
        irma_error_merging += evaluator.evaluate(irma_test[i],merge_labels(top_k_labels))
    else:
        print(i)
        irma_error_merging += evaluator.evaluate(irma_test[i],label_encoder.inverse_transform(top_k[i:(i+1),0].numpy())[0])

irma_error_merging